In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [105]:
train_dir = 'imagedata/training'
val_dir = 'imagedata/validation'
test_dir = 'imagedata/testing'

In [137]:

# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1.0/255.0,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [150]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

if test_generator.samples == 0:
    raise ValueError("Test generator has no samples. Check the test directory and ensure it has images.")

Found 2124 images belonging to 2 classes.
Found 518 images belonging to 2 classes.
Found 982 images belonging to 2 classes.


In [151]:
train_generator.class_indices

{'laptop': 0, 'mobile': 1}

In [152]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu',input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [153]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [159]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=40
)

Epoch 1/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 22s 313ms/step - accuracy: 0.8478 - loss: 0.3650 - val_accuracy: 0.8359 - val_loss: 0.3667
Epoch 2/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 307us/step - accuracy: 0.8125 - loss: 0.5639 - val_accuracy: 0.8333 - val_loss: 0.2297
Epoch 3/40


C:\Users\patel\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 306ms/step - accuracy: 0.8545 - loss: 0.3540 - val_accuracy: 0.8613 - val_loss: 0.3502
Epoch 4/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.7188 - loss: 0.4562 - val_accuracy: 0.8333 - val_loss: 0.6043
Epoch 5/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 302ms/step - accuracy: 0.8359 - loss: 0.3786 - val_accuracy: 0.8555 - val_loss: 0.3305
Epoch 6/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 155us/step - accuracy: 0.8438 - loss: 0.5069 - val_accuracy: 0.6667 - val_loss: 0.4708
Epoch 7/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 303ms/step - accuracy: 0.8625 - loss: 0.3507 - val_accuracy: 0.8750 - val_loss: 0.3128
Epoch 8/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - accuracy: 0.7188 - loss: 0.5394 - val_accuracy: 1.0000 - val_loss: 0.2726
Epoch 9/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 302ms/step - accuracy: 0.8538 - loss: 0.3389 - val_accuracy: 0.8574 - val_loss: 0.3176
Epoch 10/40
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 259us/step - accuracy: 0.9375 - loss: 0.1873 - val_accuracy: 1.0000 -

In [160]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

# Save the model
model.save(f'mobile_laptop_classifier{accuracy:.3f}.keras')

31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.9096 - loss: 0.2257
Test accuracy: 0.92


In [163]:
def predict_image(image_folder_path, model):
    predictions = []
    
    for image_name in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_name)
        
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=(200, 200))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalize the image
        
        prediction = model.predict(img_array)
        predictions.append(prediction)
        
        if prediction > 0.5:
            print(f"{image_name}: Mobile")
        else:
            print(f"{image_name}: Laptop")
    
    return predictions

# Example usage
image_folder_path = 'testing'
predictions = predict_image(image_folder_path, model)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1.jpg: Laptop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
11.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
12.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
13.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
14.jpg: Laptop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
15.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
16.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
17.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
18.jpg: Laptop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
19.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2.jpg: Laptop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
20.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
21.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
22.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
24.jpg: Mobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
25.jpg: Laptop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
26.jpg: Laptop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
27.jpg: Laptop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
28.jpg: La